# 📊 Exploração de Dados - NeuroTranslator PT-EN

Este notebook contém a análise exploratória dos datasets de tradução Português-Inglês que serão utilizados para treinar nossos modelos neurais.

## Objetivos:
- Carregar e analisar datasets paralelos PT-EN
- Estatísticas descritivas dos textos
- Análise de distribuição de comprimento
- Identificação de padrões linguísticos
- Preparação para pré-processamento

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import spacy
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Bibliotecas importadas com sucesso!")

## 1. Carregamento dos Datasets

Vamos carregar datasets públicos de tradução PT-EN disponíveis:

In [ ]:
# Download de recursos NLTK necessários
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Carregamento de modelos spaCy
try:
    nlp_pt = spacy.load('pt_core_news_sm')
    nlp_en = spacy.load('en_core_web_sm')
    print("✅ Modelos spaCy carregados com sucesso!")
except OSError:
    print("⚠️ Modelos spaCy não encontrados. Execute:")
    print("python -m spacy download pt_core_news_sm")
    print("python -m spacy download en_core_web_sm")

In [ ]:
# Função para carregar datasets de tradução
def load_translation_datasets():
    """
    Carrega datasets de tradução PT-EN de fontes públicas
    """
    datasets = {}
    
    # Dataset 1: OpenSubtitles (exemplo)
    # Aqui você pode adicionar código para carregar datasets reais
    
    # Dataset sintético para demonstração
    sample_data = {
        'portuguese': [
            "Olá, como você está?",
            "Eu gosto de programar em Python.",
            "O tempo está muito bom hoje.",
            "Vamos trabalhar juntos neste projeto.",
            "A inteligência artificial é fascinante."
        ],
        'english': [
            "Hello, how are you?",
            "I like programming in Python.",
            "The weather is very nice today.",
            "Let's work together on this project.",
            "Artificial intelligence is fascinating."
        ]
    }
    
    datasets['sample'] = pd.DataFrame(sample_data)
    
    return datasets

# Carregar datasets
datasets = load_translation_datasets()
print(f"📊 Datasets carregados: {list(datasets.keys())}")

# Visualizar amostra
sample_df = datasets['sample']
print(f"\n📈 Tamanho do dataset de exemplo: {len(sample_df)} pares")
sample_df.head()

## 2. Análise Estatística Básica

In [ ]:
def analyze_text_statistics(df):
    """
    Analisa estatísticas básicas dos textos
    """
    stats = {}
    
    for lang in ['portuguese', 'english']:
        texts = df[lang].astype(str)
        
        # Estatísticas de comprimento
        char_lengths = texts.str.len()
        word_counts = texts.str.split().str.len()
        
        stats[lang] = {
            'total_sentences': len(texts),
            'avg_char_length': char_lengths.mean(),
            'avg_word_count': word_counts.mean(),
            'max_char_length': char_lengths.max(),
            'min_char_length': char_lengths.min(),
            'std_char_length': char_lengths.std(),
            'std_word_count': word_counts.std()
        }
    
    return stats

# Analisar estatísticas
stats = analyze_text_statistics(sample_df)

# Criar DataFrame para visualização
stats_df = pd.DataFrame(stats).T
print("📊 Estatísticas dos Textos:")
stats_df.round(2)

## 3. Visualizações

In [ ]:
# Gráfico de distribuição de comprimento
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Comprimento em caracteres
pt_chars = sample_df['portuguese'].str.len()
en_chars = sample_df['english'].str.len()

axes[0, 0].hist(pt_chars, bins=20, alpha=0.7, label='Português', color='blue')
axes[0, 0].set_title('Distribuição - Caracteres (PT)')
axes[0, 0].set_xlabel('Número de Caracteres')
axes[0, 0].set_ylabel('Frequência')

axes[0, 1].hist(en_chars, bins=20, alpha=0.7, label='Inglês', color='red')
axes[0, 1].set_title('Distribuição - Caracteres (EN)')
axes[0, 1].set_xlabel('Número de Caracteres')
axes[0, 1].set_ylabel('Frequência')

# Comprimento em palavras
pt_words = sample_df['portuguese'].str.split().str.len()
en_words = sample_df['english'].str.split().str.len()

axes[1, 0].hist(pt_words, bins=20, alpha=0.7, label='Português', color='blue')
axes[1, 0].set_title('Distribuição - Palavras (PT)')
axes[1, 0].set_xlabel('Número de Palavras')
axes[1, 0].set_ylabel('Frequência')

axes[1, 1].hist(en_words, bins=20, alpha=0.7, label='Inglês', color='red')
axes[1, 1].set_title('Distribuição - Palavras (EN)')
axes[1, 1].set_xlabel('Número de Palavras')
axes[1, 1].set_ylabel('Frequência')

plt.tight_layout()
plt.show()

## 4. Análise de Vocabulário

In [ ]:
def analyze_vocabulary(texts, language='portuguese'):
    """
    Analisa o vocabulário dos textos
    """
    # Tokenização
    all_words = []
    for text in texts:
        words = word_tokenize(text.lower())
        all_words.extend([word for word in words if word.isalpha()])
    
    # Estatísticas de vocabulário
    vocab_stats = {
        'total_words': len(all_words),
        'unique_words': len(set(all_words)),
        'vocabulary_richness': len(set(all_words)) / len(all_words) if all_words else 0
    }
    
    # Palavras mais frequentes
    word_freq = Counter(all_words)
    most_common = word_freq.most_common(10)
    
    return vocab_stats, most_common, all_words

# Analisar vocabulário português
pt_vocab_stats, pt_common, pt_words = analyze_vocabulary(sample_df['portuguese'], 'portuguese')
en_vocab_stats, en_common, en_words = analyze_vocabulary(sample_df['english'], 'english')

print("🇧🇷 Estatísticas do Vocabulário Português:")
for key, value in pt_vocab_stats.items():
    print(f"  {key}: {value:.3f}")

print("\n🇺🇸 Estatísticas do Vocabulário Inglês:")
for key, value in en_vocab_stats.items():
    print(f"  {key}: {value:.3f}")

print("\n📝 Palavras mais comuns (PT):", pt_common[:5])
print("📝 Palavras mais comuns (EN):", en_common[:5])

## 5. Preparação para Modelagem

In [ ]:
# Função de pré-processamento básico
def preprocess_text(text):
    """
    Pré-processamento básico do texto
    """
    # Converter para minúsculas
    text = text.lower()
    
    # Remover caracteres especiais (manter pontuação básica)
    import re
    text = re.sub(r'[^a-záàâãéêíóôõúç\s.,!?]', '', text)
    
    # Normalizar espaços
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Aplicar pré-processamento
sample_df['portuguese_clean'] = sample_df['portuguese'].apply(preprocess_text)
sample_df['english_clean'] = sample_df['english'].apply(preprocess_text)

print("🧹 Exemplo de pré-processamento:")
print("Original (PT):", sample_df['portuguese'].iloc[0])
print("Limpo (PT):   ", sample_df['portuguese_clean'].iloc[0])
print("\nOriginal (EN):", sample_df['english'].iloc[0])
print("Limpo (EN):   ", sample_df['english_clean'].iloc[0])

## 6. Salvamento dos Dados Processados

In [ ]:
# Salvar dados processados
output_path = '../data/processed_translation_data.csv'
sample_df.to_csv(output_path, index=False, encoding='utf-8')

print(f"💾 Dados processados salvos em: {output_path}")
print(f"📊 Total de pares de tradução: {len(sample_df)}")

# Resumo final
print("\n📋 Resumo da Exploração:")
print(f"  • Dataset analisado com {len(sample_df)} pares PT-EN")
print(f"  • Comprimento médio (PT): {sample_df['portuguese'].str.len().mean():.1f} caracteres")
print(f"  • Comprimento médio (EN): {sample_df['english'].str.len().mean():.1f} caracteres")
print(f"  • Vocabulário único (PT): {len(set(' '.join(sample_df['portuguese']).split()))} palavras")
print(f"  • Vocabulário único (EN): {len(set(' '.join(sample_df['english']).split()))} palavras")
print("\n✅ Exploração de dados concluída!")

## 📝 Próximos Passos

1. **Coleta de Dados Reais**: Integrar datasets maiores como OpenSubtitles, OPUS, etc.
2. **Análise Linguística Avançada**: POS tagging, análise sintática
3. **Preparação para Treinamento**: Tokenização, criação de vocabulários
4. **Augmentação de Dados**: Técnicas para aumentar o dataset
5. **Validação Cruzada**: Divisão em treino/validação/teste

---

**Desenvolvido para o NeuroTranslator PT-EN** 🧠🔄🌐